In [1]:
from sbe_vallib import BinaryValidation, Scorer, Sampler

In [13]:
from sklearn.metrics import recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import pandas as pd


def recall_at_10(true, preds):
    return recall_score(true, preds)

def custom_test(sampler, scorer):
    pass

class CustomSampler(Sampler):
    def __init__(self, X_train, y_train, X_oos, y_oos, **kwargs):
        super().__init__(X_train, y_train, X_oos, y_oos, **kwargs)
        
        self.samplers = {
            "train": (X_train, y_train),
            "oos": (X_oos, y_oos),
        }
    

X, y = make_classification(n_samples=1000, n_classes=2,
                               n_features=5, n_informative=3, n_redundant=0,
                               random_state=0)

X = pd.DataFrame(X)
y = pd.Series(y)

X_train, X_test,  y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

model = LogisticRegression().fit(X_train, y_train)


sampler = CustomSampler(X_train, y_train, X_test, y_test)
scorer = Scorer(metrics={'f1': f1_score}, custom_metric={'recall_at_10': recall_at_10})
print(scorer.score(y_target=y_test, y_pred=model.predict(X_test)))

validator = BinaryValidation(model, sampler, scorer, custom_tests=[custom_test], exclude_list=[])

# validator.validate()

{'f1': 0.7098976109215017}
